This section seeks to provide a ranking of how efficiently teams spend their money on a yearly basis. These values will be computed by using a model to predict the salary of a given player on a given team, in a given year. This salary will be compared with the player's actual salary and the difference between the actual and the predicted will tell us if someone is overpaid or underpaid. We will then sum all of these values up for a given team and this will tell us the total amount that they are overpaying or underpaying by. We can then divide this by the total amount spent on salaries, so that we can see overpay or underpay as a percentage of total salary spent, which should make it easier to compare these values across teams and across time.

In [60]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# Load in neural network for prediction
from common import *
model = get_rf()
# model = get_nn()
nba_data = get_baseline_data("../data/cleaned_data/base_cleaned.csv")
# nba_data.loc[nba_data['playerName'] == 'Michael Jordan'][['playerName','seasonStartYear', 'team']]


c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['inflationAdjSalary_log'] = nba['inflationAdjSalary'].apply(lambda x: np.log(x))


In [62]:
# Compute the difference between the amount a team is predicted to spend v. the amount that they actually spent
COL_START_YEAR = 'seasonStartYear'
COL_TEAM = 'team'
COL_PLAYER_NAME = 'playerName'
COL_SALARY = 'inflationAdjSalary'
for year in pd.unique(nba_data[COL_START_YEAR]) :
    for team in pd.unique(nba_data.loc[nba_data[COL_START_YEAR] == year][COL_TEAM]) :
        team_year_player_data = nba_data.loc[(nba_data[COL_TEAM] == team) & (nba_data[COL_START_YEAR] == year)]
        actual_pred_dif = 0
        total_salary_spend = 0
        pred_total_salary_spend = 0
        for i, player in team_year_player_data.iterrows() :
            total_salary_spend += player[COL_SALARY]
            x = (player[get_nn_features()].values).reshape(1,-1)
            pred_val = np.exp(model.predict(x))
            pred_total_salary_spend += pred_val
            print(pred_val)
            actual_pred_dif += player[COL_SALARY] - pred_val
        print("In ", year, " team ", team, " spent ", total_salary_spend, " but should have only spent ", pred_total_salary_spend[0])
        print("Dif: ", total_salary_spend - pred_total_salary_spend[0])
    break


[15877938.72539163]
[9821257.42677664]
[5279104.51729179]
[6973196.17243656]
[10046284.06248528]
[1490977.61413296]
[1814549.96418766]
[2815415.43123716]
[2079298.57707022]
[1215170.28099388]
[1477778.91612586]
In  1996  team  CHI  spent  92432116  but should have only spent  58890971.68812963
Dif:  33541144.311870366
[21370457.12604889]
[15386141.53107192]
[11596450.84869081]
[6935443.]
[8375694.52265198]
[7027498.42682808]
[3052752.2965151]
[838016.81820371]
[1924801.41110782]
[1001023.68929477]
[808579.33718799]
In  1996  team  ORL  spent  83610754  but should have only spent  78316859.00760107
Dif:  5293894.992398933
[13539219.24212774]
[11854883.73014122]
[7756872.34028844]
[8556591.03705597]
[8338355.76000198]
[5218011.22167986]
[2810697.1647707]
[3825789.29182738]
[1781863.11977277]
[2237767.45487383]
[1238987.64202432]
[1445945.03841721]
[556412.80879496]
In  1996  team  IND  spent  67561617  but should have only spent  69161395.85177639
Dif:  -1599778.8517763913
[17359573.6953